In [27]:
import os
import sys
import openpyxl
import pygal
import pyodbc
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics import mosaicplot
from statsmodels.graphics.mosaicplot import mosaic
sns.set(style="ticks", color_codes=True)
from rpy2.robjects.packages import importr
import requests
import time
import xlrd
import warnings
warnings.filterwarnings("ignore")
plt.style.use('ggplot')

In [28]:
# conn = pyodbc.connect("Driver={SQL Server};"
#                       "Server=DESKTOP-MS8S2RN;"
#                       "Database=NCSES;"
#                       "Trusted_Connection=yes;"
# )   

# cursor = conn.cursor()
# cursor.execute("SELECT * from dbo.SED1")
# for row in cursor:
#     print(row)  

In [29]:
#  Doctorate Recipients by DemographDoctrate reciepients by Citizenship,  Race/Ethnicity, Sex, S&E Fields, Broad Fields, Detailed Fields, and Year
# Data Download from NCSES Interactive Data Tool

# Readind data from SQL Server
conn = pyodbc.connect("Driver={SQL Server};"
                      "Server=DESKTOP-MS8S2RN;"
                      "Database=NCSES;"
                      "Trusted_Connection=yes;"
)   

cursor = conn.cursor()

df1 = pd.read_sql_query('SELECT * from dbo.SED1',conn)
df1 = df1.set_index(['S&E Fields', 'Broad Fields', 'Detailed Fields'])
df1.head(3)
#df1.columns

2018  \
S&E Fields              Broad Fields  Detailed Fields                                        
Science and engineering Total         Total                                        42227.0   
                        Life sciences Total                                        12780.0   
                                      Agricultural sciences and natural resources   1445.0   

                                                                                      2017  \
S&E Fields              Broad Fields  Detailed Fields                                        
Science and engineering Total         Total                                        41294.0   
                        Life sciences Total                                        12555.0   
                                      Agricultural sciences and natural resources   1494.0   

                                                                                      2016  \
S&E Fields              Broad Fields  Detailed Fields                                        
Science and engineering Total         Total                                        41234.0   
                        Life sciences Total                                        12536.0   
                                      Agricultural sciences and natural resources   1378.0   

                                                                                      2015  \
S&E Fields              Broad Fields  Detailed Fields                                        
Science and engineering Total         Total                                        41178.0   
                        Life sciences Total                                        12493.0   
                                      Agricultural sciences and natural resources   1434.0   

                                                                                      2014  \
S&E Fields              Broad Fields  Detailed Fields                                        
Science and engineering Total         Total                                        40633.0   
                        Life sciences Total                                        12484.0   
                                      Agricultural sciences and natural resources   1338.0   

                                                                                      2013  \
S&E Fields              Broad Fields  Detailed Fields                                        
Science and engineering Total         Total                                        39031.0   
                        Life sciences Total                                        12207.0   
                                      Agricultural sciences and natural resources   1324.0   

                                                                                      2012  \
S&E Fields              Broad Fields  Detailed Fields                                        
Science and engineering Total         Total                                        37846.0   
                        Life sciences Total                                        11964.0   
                                      Agricultural sciences and natural resources   1255.0   

                                                                                      2011  \
S&E Fields              Broad Fields  Detailed Fields                                        
Science and engineering Total         Total                                        36332.0   
                        Life sciences Total                                        11535.0   
                                      Agricultural sciences and natural resources   1206.0   

                                                                                      2010  \
S&E Fields              Broad Fields  Detailed Fields                                        
Science and engineering Total         Total                                        34997.0   
                        Life science

In [30]:

df1s = df1.stack()
df1s =df1s.to_frame()
df1s.head(10)
df1s = df1s.reset_index()
df1s.columns = ['S&E Fields', 'Broad Fields', 'Detailed Fields', 'Year', 'Number']
# # df1s.columns
df1s.head()
#df1s.shape

,S&E Fields,Broad Fields,Detailed Fields,Year,Number
0,Science and engineering,Total,Total,2018,42227.0
1,Science and engineering,Total,Total,2017,41294.0
2,Science and engineering,Total,Total,2016,41234.0
3,Science and engineering,Total,Total,2015,41178.0
4,Science and engineering,Total,Total,2014,40633.0


In [34]:
df1s = df1s[df1s['Detailed Fields'] != 'Total']
df1s = df1s.reset_index()
df1s = df1s[['S&E Fields', 'Broad Fields', 'Detailed Fields', 'Year', 'Number']]
#df1s["Number"] = df1s["Number"].astype(str).astype(float)
df1s.head(3)
df1s.tail(3)
df1s.shape
#df1s.dtypes

(2060, 5)

In [32]:
# Replace no data values '-' to Nan so that it can be dropped
df1s['Number'].replace('-', np.nan, inplace=True)
df1s = df1s.dropna()
#df1s.describe
#df1s.dtypes

# Converting from object to integer
df1s["Number"] = df1s["Number"].astype(str).astype(int)
df1s.dtypes
df1s.head(3)
#df1s.shape

ValueError: invalid literal for int() with base 10: '1445.0'

In [ ]:
p1 =  df1s.groupby(['S&E Fields', 'Year'])['Number'].sum()
p1 = p1.reset_index()
p1.head(20)
p1=p1.pivot(index='Year', columns='S&E Fields', values='Number')
p1.head(40)
p1.shape
# Adding the stacked=True option to plot() 
# creates a stacked bar plot
p1.plot(kind='barh',colormap='tab20',figsize=(8, 10), width=1.0, stacked=True).legend(
    loc='lower right', ncol=1, title="S&E Fields ")
plt.title("Number of Doctrate Reciepients by Year (S&E and Non S&E)")
plt.xlabel("Number")
plt.ylabel("Year")

In [ ]:
p2 =  df1s.groupby(['Broad Fields', 'Year'])['Number'].sum()
p2 = p2.reset_index()
p2=p2.pivot(index='Year', columns='Broad Fields', values='Number')


# Adding the stacked=True option to plot() 
# creates a stacked bar plot
p2.plot(kind='barh',colormap='tab20',figsize=(10, 12), width=1.0, stacked=True).legend(
    loc='lower right', ncol=1, title="Broad Fields ")
plt.title("Number of Doctrate Reciepients by Broad Fields and Year")
plt.xlabel("Number")
plt.ylabel("Year")

In [ ]:
p3 =  df1s.groupby(['Detailed Fields', 'Year'])['Number'].sum()
p3 = p3.reset_index()
p3=p3.pivot(index='Year', columns='Detailed Fields', values='Number')


# Adding the stacked=True option to plot() 
# creates a stacked bar plot
p3.plot(kind='barh',colormap='tab20b',figsize=(15, 18), width=1.0, stacked=True).legend(
    loc='lower right', ncol=1, title="Detailed Fields ")
plt.title("Number of Doctrate Reciepients by Detailed Fields and Year")
plt.xlabel("Number")
plt.ylabel("Year")
#plt.legend(title="Line", loc='upper left', handles=handles[::-1])

In [ ]:
# df1s['S&E Fields'].unique()...
# df = df1s.groupby('S&E Fields')['Number'].nunique()
# df.count()

# df1s['Broad Fields'].unique()
# df = df1s.groupby('Broad Fields')['Number'].nunique()
# df.count()

# df1s['Detailed Fields'].unique()
# df = df1s.groupby('Detailed Fields')['Number'].nunique()
# df.count()

# df1s['Year'].unique()
# df = df1s.groupby('Year')['Number'].nunique()
# df.count()

In [ ]:
# Setting an index of three columns to conduct aggregation
#SE_NSE = df1s.set_index(['Detailed Fields'])
# SE_NSE = SE_NSE.groupby(['S&E Fields','Broad Fields', 'Year'])['Number'].sum()
# #SE_NSE=SE_NSE.sum(level='Detailed Fields')
# SE_NSE = SE_NSE.to_frame()
# SE_NSE = SE_NSE.reset_index()
# SE_NSE.head(20)
# #SE_NSE.columns
# #SE_NSE.shape

In [ ]:
#df1s.columns
#SE_NSE1.columns
SE_NSE1=df1s[['Detailed Fields', 'Year', 'Number']]

In [ ]:
SE_NSE2=SE_NSE1.pivot(index='Detailed Fields', columns='Year', values='Number')

In [ ]:
dic = SE_NSE2.to_dict('split')
#SE_NSE2.to_dict()
dic.values()
dic.keys()


In [ ]:
print(dic)
dic2 = ['index', 'data']
[dic[x] for x in dic2]

In [ ]:
dic2[0]

In [ ]:
dic['data']
#len(dic['data'])
#plt.plot(dic['data'])

In [ ]:
dic['index']
len(dic['index'])

In [ ]:
dic['columns']
len(dic['columns'])

In [ ]:
dic2 = dic['index'] + dic['data']
dic

In [ ]:
for key in dic.keys():
  print(key)

In [ ]:
type(dic['index'])

In [ ]:
df = pd.DataFrame({'col1': [1, 2],
...                    'col2': [0.5, 0.75]},
...                   index=['a', 'b'])
df.columns

In [ ]:
df.head()

In [ ]:
dic= df.to_dict()
dic.keys()

In [ ]:
df.to_dict('split')


In [ ]:

df = SE_NSE1.set_index('Year')
#print(df.T.to_dict(orient='list'))
print(df)

In [ ]:
# Doctorate Recipients by Demographic Characteristics ---- Data by Citizenship,  Race/Ethnicity, Sex, S&E Fields, Broad Fields, Detailed Fields, and Year
# Data Download from NCSES Interactive Data Tool
df2 = pd.read_csv('C:/Users/Muluken/WorkingFiles/NSF_NCSES/Data/SED_Export_2020-06-17T23_29_53.896Z.csv', skiprows=[0], header=[1])
df2 = df2.set_index(['Citizenship','Race and Ethnicity', 'Sex','S&E Fields', 'Broad Fields', 'Detailed Fields'])
df2.head()

In [ ]:
df2s = df2.stack()
df2s =df2s.to_frame()

df2s = df2s.reset_index()
df2s.columns = ['Citizenship','Race and Ethnicity', 'Sex','S&E Fields', 'Broad Fields', 'Detailed Fields', 'Year', 'Number']
df2s.head(10)

In [ ]:
df2s = df2s[df2s['Detailed Fields'] != 'Total']
df2s = df2s.reset_index()
df2s = df2s[['Citizenship','Race and Ethnicity', 'Sex','S&E Fields', 'Broad Fields', 'Detailed Fields', 'Year', 'Number']]
df2s.head()